# Input

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import xpress as xp
xp.controls.outputlog = 0

In [2]:
T = 168

In [3]:
N = 100

In [4]:
bornes_random = [np.sort(np.random.randint(low=0,high=10,size=(T,2)),axis=1) for i in range(N)]
# for i in range(N):
#     bornes_random[i][:,0] = np.zeros(T)

In [5]:
demand_random = [np.random.rand(T)*500 for i in range(N)]

In [6]:
p_min_max = [np.sort(np.random.rand(2))*50 for i in range(N)]

In [7]:
delta = [randint(1,40) for i in range(N)]

In [8]:
cost = [np.random.rand(4) for i in range(N)]

In [9]:
def main_problem(on_min, on_max, d, demand,p_min, p_max,c_f,c_v,c_u,c_s):
    p = xp.problem()
    p.controls.xslp_log = -1

    n = [xp.var(lb=on_min[i],ub=on_max[i]) for i in range(T)]
    n_plus = [xp.var(lb=0) for i in range(T)]
    n_moins = [xp.var(lb=0) for i in range(T)]
    n_moins_moins = [xp.var(lb=0,ub=max(0,on_max[i-1]-on_max[i])) for i in range(T)]
    unsupplied = [xp.var(lb=0) for i in range(T)]
    prod = [xp.var(lb=0) for i in range(T)]


    p.addVariable(n,n_plus,n_moins,n_moins_moins,unsupplied,prod)

    p.addConstraint(n[i]-n[i-1] == n_plus[i]-n_moins[i] for i in range(1,T))
    p.addConstraint(n_moins[i]>=n_moins_moins[i] for i in range(T))
    p.addConstraint(n[i]>=xp.Sum([n_plus[k]-n_moins_moins[k] for k in range(i-d+1,i+1)]) for i in range(T))
    p.addConstraint(n[i]<=on_max[i - d] - xp.Sum([n_moins[k]-max(0,on_max[k]-on_max[k-1]) for k in range(i-d+1,i+1)]) for i in range(T))
    p.addConstraint(unsupplied[i]>=demand[i]-prod[i] for i in range(T))
    p.addConstraint(prod[i]>=n[i]*p_min for i in range(T))
    p.addConstraint(prod[i]<=n[i]*p_max for i in range(T))

    p.setObjective(xp.Sum([c_u*unsupplied[i]+c_f*n[i]+c_v*prod[i]+c_s*n_plus[i] for i in range(T)]))

    p.solve()

    assert p.getProbStatus()==1

    return p.getSolution(n,n_plus,n_moins,n_moins_moins)

In [32]:
def heuristic_problem(on_min, on_max, d,debug=False,on_opt=None):
    p = xp.problem()
    p.controls.xslp_log = -1

    n = [xp.var(name=f"n_{i}",lb=on_min[i],ub=on_max[i]) for i in range(T)]
    n_plus = [xp.var(name=f"n_plus_{i}",lb=0) for i in range(T)]
    n_moins = [xp.var(name=f"n_moins_{i}",lb=0) for i in range(T)]
    n_moins_moins = [xp.var(name=f"n__moins_moins{i}",lb=0,ub=max(0,on_max[i-1]-on_max[i])) for i in range(T)]

    p.addVariable(n,n_plus,n_moins,n_moins_moins)

    if not np.all(on_opt==None):
        p.addConstraint(n[i]==on_opt[i] for i in range(T))

    p.addConstraint(n[i]-n[i-1] == n_plus[i]-n_moins[i] for i in range(1,T))
    p.addConstraint(n_moins[i]>=n_moins_moins[i] for i in range(T))
    p.addConstraint(n[i]>=xp.Sum([n_plus[k]-n_moins_moins[k] for k in range(i-d+1,i+1)]) for i in range(T))
    p.addConstraint(n[i]<=on_max[i - d] - xp.Sum([n_moins[k]-max(0,on_max[k]-on_max[k-1]) for k in range(i-d+1,i+1)]) for i in range(T))

    p.setObjective(xp.Sum([n[i] for i in range (T)]))

    if debug:
        p.write("p", "lp")

    p.solve()

    assert p.getProbStatus()==1

    return p.getSolution(n,n_plus,n_moins,n_moins_moins)

# Itération 1

In [24]:
itr_1 = []
for i in range(N):
    itr_1.append(main_problem(bornes_random[i][:,0],bornes_random[i][:,1],delta[i], demand_random[i],p_min_max[i][0],p_min_max[i][1],cost[i][0],cost[i][1],cost[i][2],cost[i][3]))

In [25]:
for i in range(N):
    for j in range(4):
        assert np.all(np.abs(np.round(itr_1[i][j])-itr_1[i][j])<=1e-5)

AssertionError: 

In [26]:
print(i, j )

1 0


In [27]:
np.where(np.abs(np.round(itr_1[i][j])-itr_1[i][j])>1e-5)

(array([  1,   2,  13,  17,  21,  22,  33,  34,  38,  39,  43,  53,  71,
         76, 113, 114, 126, 150, 151, 153, 154, 155, 165, 167], dtype=int64),)

In [28]:
itr_1[i][j][10:20]

[9.0, 6.0, 5.0, 3.631210421652427, 8.0, 2.0, 4.0, 2.9378503736771613, 5.0, 5.0]

# Ceil itération 1

In [33]:
contraintes_respectées = [heuristic_problem(np.ceil(np.round(itr_1[i][0],12)),bornes_random[i][:,1],delta[i],on_opt=np.ceil(np.round(itr_1[i][0],12))) for i in range(N)]

AssertionError: 

In [34]:
print(i)

1


# Heuristique

In [35]:
heuristic = [heuristic_problem(np.ceil(np.round(itr_1[i][0],12)),bornes_random[i][:,1],delta[i]) for i in range(N)]

In [36]:
for i in range(N):
    for j in range(4):
        assert np.all(np.abs(np.round(heuristic[i][j])-heuristic[i][j])<=1e-5)

# Heuristique sur aléatoire

In [37]:
heuristic_on_random = [heuristic_problem(bornes_random[i][:,0],bornes_random[i][:,1],delta[i]) for i in range(N)]

In [38]:
for i in range(N):
    for j in range(4):
        assert np.all(np.abs(np.round(heuristic_on_random[i][j])-heuristic_on_random[i][j])<=1e-5)

AssertionError: 

In [39]:
print(i, j)

43 0


In [40]:
np.where(np.abs(np.round(heuristic_on_random[i][j])-heuristic_on_random[i][j])>1e-5)

(array([166], dtype=int64),)

In [41]:
heuristic_on_random[i][j][165:]

[6.0, 3.5, 4.0]